**Demo for `teneva.core.opt`**

---

This module contains the function "opt_tt" which estimate the multi-indices of min and max of TT-tensor.

## Loading and importing modules

In [1]:
import numpy as np
import teneva
from time import perf_counter as tpc
np.random.seed(42)

## Function `opt_tt`

Find multi-indices which relate to min and max elements of TT-tensor.

In [2]:
n = [ 20,  18,  16,  14,  12]     # Shape of the tensor
Y = teneva.rand(n, r=4)           # Random TT-tensor with rank 4
i_min, i_max = teneva.opt_tt(Y)   # Multi-indices of min and max

print(f'i min appr :', i_min)
print(f'i max appr :', i_max)
print(f'y min appr : {teneva.get(Y, i_min):-12.4e}')
print(f'y max appr : {teneva.get(Y, i_max):-12.4e}')

i min appr : [14  6 15 13  8]
i max appr : [14  6  1 13  8]
y min appr :  -2.7379e+02
y max appr :   3.1204e+02


Let check the result:

In [3]:
Y_full = teneva.full(Y)           # Build tensor in full format
i_min = np.argmin(Y_full)         # Multi-indices of min and max
i_max = np.argmax(Y_full)

i_min = np.unravel_index(i_min, n)
i_max = np.unravel_index(i_max, n)

print(f'i min real :', i_min)
print(f'i max real :', i_max)
print(f'y min real : {Y_full[i_min]:-12.4e}')
print(f'y max real : {Y_full[i_max]:-12.4e}')

i min real : (14, 6, 15, 13, 8)
i max real : (14, 6, 1, 13, 8)
y min real :  -2.7379e+02
y max real :   3.1204e+02


We can check results for many random TT-tensors:

In [4]:
n = [ 20,  18,  16,  14,  12]    # Shape of the tensor

for i in range(10):
    Y = teneva.rand(n, r=4)
    t = tpc()
    i_min_appr, i_max_appr = teneva.opt_tt(Y)
    y_min_appr = teneva.get(Y, i_min_appr)
    y_max_appr = teneva.get(Y, i_max_appr)
    t = tpc() - t

    Y_full = teneva.full(Y)
    i_min_real = np.unravel_index(np.argmin(Y_full), n)
    i_max_real = np.unravel_index(np.argmax(Y_full), n)
    y_min_real = Y_full[i_min_real]
    y_max_real = Y_full[i_max_real]
    
    e_min = np.abs(y_min_appr - y_min_real) / np.abs(y_min_real)
    e_max = np.abs(y_max_appr - y_max_real) / np.abs(y_max_real)

    print(f'-> Error for min {e_min:-7.1e} | Error for max {e_max:-7.1e} | Time {t:-8.4f}')

-> Error for min 2.0e-16 | Error for max 1.8e-16 | Time  14.5012
-> Error for min 3.5e-16 | Error for max 1.4e-16 | Time  15.2316
-> Error for min 1.7e-16 | Error for max 1.5e-16 | Time  14.6780
-> Error for min 1.8e-16 | Error for max 1.6e-16 | Time  14.5652
-> Error for min 2.3e-16 | Error for max 0.0e+00 | Time  14.6821
-> Error for min 2.2e-16 | Error for max 1.2e-16 | Time  15.5223
-> Error for min 1.2e-16 | Error for max 0.0e+00 | Time  14.2446
-> Error for min 0.0e+00 | Error for max 0.0e+00 | Time  13.6671
-> Error for min 0.0e+00 | Error for max 0.0e+00 | Time  13.7663
-> Error for min 1.7e-16 | Error for max 1.9e-16 | Time  14.2239


We can also check it for real data (we build TT-tensor using TT-SVD here). Note that we shift all functions up by $0.5$ to ensure that its min/max values are nonzero, since we compute the relative error for result.

In [5]:
d = 6   # Dimension
n = 16  # Grid size

for func in teneva.func_demo_all(d, dy=0.5):
    # Set the uniform grid:
    func.set_grid(n, kind='uni')
    
    # Build full tensor on the grid:
    I_full = teneva.grid_flat(func.n)
    Y_full_real = func.get_f_ind(I_full).reshape(func.n, order='F')

    # Build TT-approximation by TT-SVD:
    Y = teneva.svd(Y_full_real, e=1.E-8)
    Y = teneva.truncate(Y, e=1.E-8)
    r = teneva.erank(Y)

    # Compute the exact min and max for TT-tensor:
    Y_full = teneva.full(Y)
    y_min_real = np.min(Y_full)
    y_max_real = np.max(Y_full)

    # Find the minimum and maximum of TT-tensor by opt_tt:
    t = tpc()
    i_min_appr, i_max_appr = teneva.opt_tt(Y)
    y_min_appr = teneva.get(Y, i_min_appr)
    y_max_appr = teneva.get(Y, i_max_appr)
    t = tpc() - t
    
    # Check the accuracy of result:
    e_min = abs((y_min_real - y_min_appr) / y_min_real)
    e_max = abs((y_max_real - y_max_appr) / y_max_real)
    
    # Present the result:
    text = '-> ' + func.name + ' ' * max(0, 20 - len(func.name)) + ' | '
    text += f'TT-rank {r:-5.1f} | '
    text += f'Error for min {e_min:-7.1e} | '
    text += f'Error for max {e_max:-7.1e} | '
    text += f'Time {t:-8.4f} | '
    print(text)

-> Ackley               | TT-rank   9.4 | Error for min 2.0e-16 | Error for max 0.0e+00 | Time   8.5492 | 
-> Alpine               | TT-rank   2.0 | Error for min 7.7e-16 | Error for max 0.0e+00 | Time   8.2908 | 
-> Brown                | TT-rank  10.3 | Error for min 1.9e-07 | Error for max 1.6e-15 | Time  10.4127 | 
-> Dixon                | TT-rank   6.4 | Error for min 1.0e-12 | Error for max 0.0e+00 | Time  14.0481 | 
-> Exponential          | TT-rank   2.0 | Error for min 1.1e-16 | Error for max 0.0e+00 | Time   7.4840 | 
-> Grienwank            | TT-rank   4.2 | Error for min 1.0e-15 | Error for max 0.0e+00 | Time  11.5836 | 
-> Michalewicz          | TT-rank   2.0 | Error for min 0.0e+00 | Error for max 0.0e+00 | Time   7.8507 | 
-> Qing                 | TT-rank   6.0 | Error for min 4.6e-08 | Error for max 1.6e-16 | Time   9.2934 | 
-> Rastrigin            | TT-rank   4.6 | Error for min 4.8e-16 | Error for max 0.0e+00 | Time   9.0017 | 
-> Rosenbrock           | TT-rank   6

We can also check it for real data with TT-CROSS approach:

In [6]:
d = 6   # Dimension
n = 16  # Grid size

for func in teneva.func_demo_all(d, dy=0.5):
    # Set the uniform grid:
    func.set_grid(n, kind='uni')

    # Build TT-approximation by TT-CROSS:
    Y = teneva.rand(func.n, r=1)
    Y = teneva.cross(func.get_f_ind, Y, m=1.E+5, dr_max=1, cache={})
    Y = teneva.truncate(Y, e=1.E-8)
    r = teneva.erank(Y)

    # Compute the exact min and max for TT-tensor:
    Y_full = teneva.full(Y)
    y_min_real = np.min(Y_full)
    y_max_real = np.max(Y_full)
    
    # Find the minimum and maximum of TT-tensor by opt_tt:
    t = tpc()
    i_min_appr, i_max_appr = teneva.opt_tt(Y)
    y_min_appr = teneva.get(Y, i_min_appr)
    y_max_appr = teneva.get(Y, i_max_appr)
    t = tpc() - t
    
    # Check the accuracy of result:
    e_min = abs((y_min_real - y_min_appr) / y_min_real)
    e_max = abs((y_max_real - y_max_appr) / y_max_real)
    
    # Present the result:
    text = '-> ' + func.name + ' ' * max(0, 20 - len(func.name)) + ' | '
    text += f'TT-rank {r:-5.1f} | '
    text += f'Error for min {e_min:-7.1e} | '
    text += f'Error for max {e_max:-7.1e} | '
    text += f'Time {t:-8.4f} | '
    print(text)

-> Ackley               | TT-rank  11.6 | Error for min 0.0e+00 | Error for max 0.0e+00 | Time   7.8370 | 
-> Alpine               | TT-rank   5.9 | Error for min 0.0e+00 | Error for max 0.0e+00 | Time   9.2351 | 
-> Brown                | TT-rank  11.0 | Error for min 8.2e-10 | Error for max 1.4e-15 | Time   9.9420 | 
-> Dixon                | TT-rank   7.6 | Error for min 1.9e-12 | Error for max 2.4e-16 | Time  14.0661 | 
-> Exponential          | TT-rank   6.5 | Error for min 0.0e+00 | Error for max 0.0e+00 | Time   8.5580 | 
-> Grienwank            | TT-rank   5.4 | Error for min 7.7e-15 | Error for max 0.0e+00 | Time  11.5098 | 
-> Michalewicz          | TT-rank   6.7 | Error for min 0.0e+00 | Error for max 5.1e-15 | Time   8.8541 | 
-> Qing                 | TT-rank   9.3 | Error for min 9.8e-09 | Error for max 3.3e-16 | Time   9.7634 | 
-> Rastrigin            | TT-rank   4.4 | Error for min 4.0e-15 | Error for max 1.2e-16 | Time   9.2471 | 
-> Rosenbrock           | TT-rank   3

We can also log the optimization process (note that for the Rosenbrock function it takes quite a lot of iterations to get a good minimum and maximum value):

In [7]:
func = teneva.FuncDemoRosenbrock(d=6, dy=0.5)
func.set_grid(n=16, kind='uni')

Y = teneva.rand(func.n, r=1)
Y = teneva.cross(func.get_f_ind, Y, m=1.E+5, dr_max=1, cache={})
Y = teneva.truncate(Y, e=1.E-8)

Y_full = teneva.full(Y)
y_min = np.min(Y_full)
y_max = np.max(Y_full)
print(f'Real values for TT-tensor        | y_min = {y_min:-16.7e} | y_max = {y_max:-16.7e} |\n')
    
i_min_appr, i_max_appr = teneva.opt_tt(Y, log=True)

Real values for TT-tensor        | y_min =    1.4047443e+00 | y_max =    1.9530131e+04 |

outer : pre | ... | rank =   3.7 | y_min =    1.2690465e+01 | y_max =    1.9530131e+04 | 
outer :   1 | ... | 
inner :   0 | MIN | rank =   4.7 | y_min =    1.2690465e+01 | y_max =    1.9530131e+04 | y_eps =    0.0000000e+00 | 
inner :   1 | MIN | rank =   6.0 | y_min =    9.4878651e+00 | y_max =    1.9530131e+04 | y_eps =    3.2025997e+00 | 
inner :   2 | MIN | rank =  13.1 | y_min =    9.4878651e+00 | y_max =    1.9530131e+04 | y_eps =    0.0000000e+00 | 
inner :   3 | MIN | rank =  18.6 | y_min =    9.4878651e+00 | y_max =    1.9530131e+04 | y_eps =    0.0000000e+00 | 
inner :   4 | MIN | rank =  22.6 | y_min =    1.4047443e+00 | y_max =    1.9530131e+04 | y_eps =    8.0831208e+00 | 
inner :   0 | MAX | rank =   4.7 | y_min =    1.4047443e+00 | y_max =    1.9530131e+04 | y_eps =    0.0000000e+00 | 
inner :   1 | MAX | rank =   5.9 | y_min =    1.4047443e+00 | y_max =    1.9530131e+04 | y_eps = 

Additionally, we can carry out a more complicated check for essentially multidimensional tensors. We generate a random TT-tensor and manually set its minimum value:

In [9]:
y_min_real  = 1.      # Value for min
y_min_scale = 2.      # Scale

def delta(n, cind=0):
    d = len(n)
    Y = []
    for i in range(d):
        G = np.zeros((1, n[i], 1))
        G[0, cind[i], 0] = 1
        Y.append(G)
    return Y

for d in range(3, 11):    # Dimension of the tensor
    n = [20] * d          # Shape of the tensor
    r = 2                 # TT-rank of the tensor
    i_min_real = [2] * d  # Multi-index for min
    

    for i in range(3):
        Y = teneva.rand(n, r)
        Y = teneva.mul(Y, Y)
        Y = teneva.add(Y, y_min_real * y_min_scale)
        y = teneva.get(Y, i_min_real)
        D = delta(n, i_min_real)
        D = teneva.mul(D, y_min_real - y)
        Y = teneva.add(Y, D)
        Y = teneva.truncate(Y, 1.E-16)

        y_min_real = teneva.get(Y, i_min_real)
        r_real = teneva.erank(Y)

        t = tpc()
        i_min_appr, i_max_appr = teneva.opt_tt(Y, nswp_outer=2, nswp=20, r=50, e=1.E-8, log=False)
        y_min_appr = teneva.get(Y, i_min_appr)
        y_max_appr = teneva.get(Y, i_max_appr)
        t = tpc() - t

        e_min = np.abs((y_min_appr - y_min_real) / y_min_real)

        print(f'Dim {d:-2d} | Rank {r_real:-5.1f} | Error for min {e_min:-7.1e} | Time {t:-8.4f}')
    print()

Dim  3 | Rank   5.5 | Error for min 0.0e+00 | Time  26.7064
Dim  3 | Rank   5.5 | Error for min 0.0e+00 | Time  28.1536
Dim  3 | Rank   6.0 | Error for min 0.0e+00 | Time  26.6550

Dim  4 | Rank   5.7 | Error for min 0.0e+00 | Time  71.4249
Dim  4 | Rank   5.5 | Error for min 0.0e+00 | Time  70.1516
Dim  4 | Rank   5.3 | Error for min 0.0e+00 | Time  60.2636

Dim  5 | Rank   5.4 | Error for min 0.0e+00 | Time  72.1397
Dim  5 | Rank   5.6 | Error for min 0.0e+00 | Time  91.2491
Dim  5 | Rank   5.2 | Error for min 0.0e+00 | Time 234.8456

Dim  6 | Rank   5.2 | Error for min 0.0e+00 | Time 266.9129
Dim  6 | Rank   5.5 | Error for min 0.0e+00 | Time 238.7489
Dim  6 | Rank   5.6 | Error for min 0.0e+00 | Time 122.5100

Dim  7 | Rank   5.7 | Error for min 0.0e+00 | Time  98.4463
Dim  7 | Rank   5.4 | Error for min 0.0e+00 | Time 318.2259
Dim  7 | Rank   5.7 | Error for min 0.0e+00 | Time 604.1358

Dim  8 | Rank   5.6 | Error for min 0.0e+00 | Time 411.4440
Dim  8 | Rank   5.4 | Error for min

Let's check that the method gives (almost) always a great result. We check optimizer for 100 random TT-tensors and display the result for cases in which the error turned out to be greater than $10^{-10}$ for min or max:

We can also investigate the dependency on parameters of the method:

In [23]:
n      = [ 20,  18,  16,  14,  12]                   # Shape of the tensor
Y_list = [teneva.rand(n, r=4) for _ in range(1000)]  # A list of random TT-tensors
e_bad  = 1.E-10                                      # Error threshold for bad result

In [24]:
# What if we do only preiteration (only maxvol for original tensor):
ind_bad = []

t = tpc()

for ind in range(len(Y_list)):
    Y = Y_list[ind]
    i_min_appr, i_max_appr = teneva.opt_tt(Y, nswp_outer=0)
    y_min_appr = teneva.get(Y, i_min_appr)
    y_max_appr = teneva.get(Y, i_max_appr)

    Y_full = teneva.full(Y)
    i_min_real = np.unravel_index(np.argmin(Y_full), n)
    i_max_real = np.unravel_index(np.argmax(Y_full), n)
    y_min_real = Y_full[i_min_real]
    y_max_real = Y_full[i_max_real]
    
    e_min = np.abs(y_min_appr - y_min_real) / np.abs(y_min_real)
    e_max = np.abs(y_max_appr - y_max_real) / np.abs(y_max_real)

    if e_min > e_bad or e_max > e_bad:
        ind_bad.append(ind)
        # print(f'Bad case (# {ind+1:-5d}). -> Error for min {e_min:-7.1e} | Error for max {e_max:-7.1e}')
        
k_all = len(Y_list)
k_bad = len(ind_bad)
t = (tpc() - t) / k_all if k_all else 0.

print('-' * 70)
print(f'Average time    : {t:-8.4f}')
print(f'Total bad cases : {k_bad:-8d}')

----------------------------------------------------------------------
Average time    :   0.2860
Total bad cases :      515


In [25]:
# Let consider bad results and do only "Y-y_ref" operation:
ind_bad_2 = []

t = tpc()

for ind in ind_bad:
    Y = Y_list[ind]
    i_min_appr, i_max_appr = teneva.opt_tt(Y, nswp_outer=1, nswp=0)
    y_min_appr = teneva.get(Y, i_min_appr)
    y_max_appr = teneva.get(Y, i_max_appr)

    Y_full = teneva.full(Y)
    i_min_real = np.unravel_index(np.argmin(Y_full), n)
    i_max_real = np.unravel_index(np.argmax(Y_full), n)
    y_min_real = Y_full[i_min_real]
    y_max_real = Y_full[i_max_real]
    
    e_min = np.abs(y_min_appr - y_min_real) / np.abs(y_min_real)
    e_max = np.abs(y_max_appr - y_max_real) / np.abs(y_max_real)

    if e_min > e_bad or e_max > e_bad:
        ind_bad_2.append(ind)
        print(f'Bad case (# {ind+1:-5d}). -> Error for min {e_min:-7.1e} | Error for max {e_max:-7.1e}')
        
k_all = len(ind_bad)
k_bad = len(ind_bad_2)
t = (tpc() - t) / k_all if k_all else 0.

print('-' * 70)
print(f'Average time    : {t:-8.4f}')
print(f'Total bad cases : {k_bad:-8d}')

Bad case (#     2). -> Error for min 0.0e+00 | Error for max 1.3e-01
Bad case (#     8). -> Error for min 0.0e+00 | Error for max 7.3e-02
Bad case (#     9). -> Error for min 0.0e+00 | Error for max 1.0e-01
Bad case (#    10). -> Error for min 1.6e-01 | Error for max 1.7e-01
Bad case (#    11). -> Error for min 7.5e-03 | Error for max 2.8e-02
Bad case (#    12). -> Error for min 8.2e-02 | Error for max 1.6e-16
Bad case (#    14). -> Error for min 0.0e+00 | Error for max 2.4e-02
Bad case (#    17). -> Error for min 4.6e-02 | Error for max 0.0e+00
Bad case (#    18). -> Error for min 2.1e-01 | Error for max 0.0e+00
Bad case (#    21). -> Error for min 2.2e-16 | Error for max 3.5e-02
Bad case (#    26). -> Error for min 1.1e-01 | Error for max 0.0e+00
Bad case (#    28). -> Error for min 7.3e-02 | Error for max 1.0e-01
Bad case (#    29). -> Error for min 4.5e-02 | Error for max 4.7e-02
Bad case (#    30). -> Error for min 1.8e-01 | Error for max 1.7e-02
Bad case (#    31). -> Error for m

In [26]:
# Let consider bad results and do only one inner iteration
# (i.e., maxvol for "(Y-y_ref)^2"):
ind_bad_3 = []

t = tpc()

for ind in ind_bad_2:
    Y = Y_list[ind]
    i_min_appr, i_max_appr = teneva.opt_tt(Y, nswp_outer=1, nswp=1, r=100)
    y_min_appr = teneva.get(Y, i_min_appr)
    y_max_appr = teneva.get(Y, i_max_appr)

    Y_full = teneva.full(Y)
    i_min_real = np.unravel_index(np.argmin(Y_full), n)
    i_max_real = np.unravel_index(np.argmax(Y_full), n)
    y_min_real = Y_full[i_min_real]
    y_max_real = Y_full[i_max_real]
    
    e_min = np.abs(y_min_appr - y_min_real) / np.abs(y_min_real)
    e_max = np.abs(y_max_appr - y_max_real) / np.abs(y_max_real)

    if e_min > e_bad or e_max > e_bad:
        ind_bad_3.append(ind)
        print(f'Bad case (# {ind+1:-5d}). -> Error for min {e_min:-7.1e} | Error for max {e_max:-7.1e}')
              
k_all = len(ind_bad_2)
k_bad = len(ind_bad_3)
t = (tpc() - t) / k_all if k_all else 0.

print('-' * 70)
print(f'Average time    : {t:-8.4f}')
print(f'Total bad cases : {k_bad:-8d}')

Bad case (#    37). -> Error for min 1.2e-16 | Error for max 1.9e-04
Bad case (#   116). -> Error for min 3.8e-02 | Error for max 2.1e-16
Bad case (#   128). -> Error for min 4.2e-02 | Error for max 0.0e+00
Bad case (#   164). -> Error for min 1.1e-16 | Error for max 7.7e-03
Bad case (#   254). -> Error for min 0.0e+00 | Error for max 7.4e-03
Bad case (#   361). -> Error for min 4.1e-03 | Error for max 1.2e-16
Bad case (#   382). -> Error for min 0.0e+00 | Error for max 3.0e-02
Bad case (#   387). -> Error for min 1.3e-16 | Error for max 3.0e-02
Bad case (#   451). -> Error for min 0.0e+00 | Error for max 1.3e-02
Bad case (#   512). -> Error for min 6.6e-03 | Error for max 0.0e+00
Bad case (#   562). -> Error for min 1.2e-16 | Error for max 1.1e-01
Bad case (#   621). -> Error for min 1.4e-16 | Error for max 1.4e-02
Bad case (#   650). -> Error for min 1.2e-16 | Error for max 3.8e-02
Bad case (#   664). -> Error for min 0.0e+00 | Error for max 5.3e-02
Bad case (#   903). -> Error for m

In [27]:
# Let consider bad results and do two inner iterations
# (i.e., maxvol for "(Y-y_ref)^2" and then for "(Y-y_ref)^4"):
ind_bad_4 = []

t = tpc()

for ind in ind_bad_3:
    Y = Y_list[ind]
    i_min_appr, i_max_appr = teneva.opt_tt(Y, nswp_outer=1, nswp=2, r=100)
    y_min_appr = teneva.get(Y, i_min_appr)
    y_max_appr = teneva.get(Y, i_max_appr)

    Y_full = teneva.full(Y)
    i_min_real = np.unravel_index(np.argmin(Y_full), n)
    i_max_real = np.unravel_index(np.argmax(Y_full), n)
    y_min_real = Y_full[i_min_real]
    y_max_real = Y_full[i_max_real]
    
    e_min = np.abs(y_min_appr - y_min_real) / np.abs(y_min_real)
    e_max = np.abs(y_max_appr - y_max_real) / np.abs(y_max_real)

    if e_min > e_bad or e_max > e_bad:
        ind_bad_4.append(ind)
        print(f'Bad case (# {ind+1:-5d}). -> Error for min {e_min:-7.1e} | Error for max {e_max:-7.1e}')
              
k_all = len(ind_bad_3)
k_bad = len(ind_bad_4)
t = (tpc() - t) / k_all if k_all else 0.

print('-' * 70)
print(f'Average time    : {t:-8.4f}')
print(f'Total bad cases : {k_bad:-8d}')

----------------------------------------------------------------------
Average time    :   7.5682
Total bad cases :        0


---